## Conversation Memory and Integration with Calendar APIs 

**Author**: Derek Rosenzweig

In this notebook, we convert structured memory traces (stored as `.json`) into `.ics` calendar files, enabling developers, users, and agents to visualize and revisit key events — particularly conversation history with language models — within their preferred calendar tools (Apple Calendar, Google Calendar, Outlook). The goal is to make memory traces both queryable and actionable by aligning them with familiar, time-based interfaces that users trust and regularly engage with.

### Use Cases for Calendar-Based Memory

By ingesting `.ics` files from tools like Google Calendar or Apple Calendar, agents can:

- Ground responses in the user’s schedule, conversation history, and events 
- Retrieve relevant context by aligning timestamps and event metadata  
- Prioritize upcoming tasks or recall previous interactions based on calendar entries  

This workflow supports persistent, time-aware context grounding—enabling language models to condition model outputs based on the user routines, workflows, and conversation history.

### What This Example Covers

- Representing memory traces (i.e., conversation history) in structured `.json` format  
- Converting traces into `.ics` format for calendar input  
- Visualizing `.ics` files in familiar calendar interfaces  
- Use cases for planning, agent orchestration, and scheduling


### 1. Define Memory Traces from Conversation History

Here's a sample `memory_traces` dictionary with a user ID, session ID, and a list of timestamped memory events. Each trace includes a type, title, and content. Optional fields like `location` can provide additional context.


### 1. Define Memory Traces from Conversation History 


Below is a sample memory_traces dictionary. It contains a user ID, a session ID, and a list of timestamped memory events—each with a type, title, and content. The `location` field is optional and can be used to provide spatial context.



In [1]:
# Example conversation history memory traces 

memory_traces = {
    "user_id": "derek_dev",
    "session_id": "cookbook_walkthrough",
    "memory": [
        {
            "id": "m101",
            "type": "goal",
            "timestamp": "2025-04-22T17:00:00Z",
            "title": "Design JSON Schema for Memory Traces",
            "content": "Finalize the structured schema to capture conversation goals, reflections, and observations.",
            "location": "SF Office"
        },
        {
            "id": "m102",
            "type": "reflection",
            "timestamp": "2025-04-23T17:00:00Z",
            "title": "Initial Demo Success",
            "content": "Successfully generated ICS from memory traces and confirmed Apple Calendar compatibility.",
            "location": "Remote Workspace"
        },
        {
            "id": "m103",
            "type": "goal",
            "timestamp": "2025-04-24T17:00:00Z",
            "title": "Send Cookbook to Collaborators",
            "content": "Package and share demo with OpenAI and Google collaborators for feedback."
        }
    ]
}

print(memory_traces)

{'user_id': 'derek_dev', 'session_id': 'cookbook_walkthrough', 'memory': [{'id': 'm101', 'type': 'goal', 'timestamp': '2025-04-22T17:00:00Z', 'title': 'Design JSON Schema for Memory Traces', 'content': 'Finalize the structured schema to capture conversation goals, reflections, and observations.', 'location': 'SF Office'}, {'id': 'm102', 'type': 'reflection', 'timestamp': '2025-04-23T17:00:00Z', 'title': 'Initial Demo Success', 'content': 'Successfully generated ICS from memory traces and confirmed Apple Calendar compatibility.', 'location': 'Remote Workspace'}, {'id': 'm103', 'type': 'goal', 'timestamp': '2025-04-24T17:00:00Z', 'title': 'Send Cookbook to Collaborators', 'content': 'Package and share demo with OpenAI and Google collaborators for feedback.'}]}


### Field Definitions

| Field             | Type     | Required | Description |
|------------------|----------|----------|-------------|
| id               | string   | yes      | Unique trace ID (deterministic or UUID). |
| type             | enum     | yes      | One of: goal, observation, reflection, event. |
| timestamp        | ISO8601  | yes      | UTC-formatted timestamp of the trace. |
| title            | string   | optional | Summary, used for calendar event summary. |
| content          | string   | yes      | Full description of the trace. |
| location         | string   | optional | Physical or contextual location. |
| url              | string   | optional | Link to a related page or resource. |
| importance       | float    | optional | Normalized value [0.0, 1.0] for prioritization. |
| completion_status| enum     | optional | pending, completed, skipped. |


### Input Modalities

| Source           | Example                             | Processing                                      |
|------------------|-------------------------------------|-------------------------------------------------|
| Chat (LLM)       | “Remind me to reorder pipette tips” | Parsed into goal with generated title and timestamp. |
| System Logs      | Uber ride at 3:45 PM                | Enriched with location, vendor, and inferred purpose. |
| Calendar Import  | Meeting invite                      | Mapped directly to event type. |



### 2. Convert JSON to `.ics`

The function below converts each memory trace into a valid calendar event in [iCalendar (`.ics`)](https://datatracker.ietf.org/doc/html/rfc5545) format.

This enables manual import into calendar apps like Google Calendar or Apple Calendar using a standards-compliant `.ics` file.


In [2]:
from datetime import datetime, timedelta
from pathlib import Path

# Function to convert ISO datetime to iCalendar format
def datetime_to_ics(dt: str) -> str:
    """Convert ISO 8601 datetime string to iCalendar format."""
    return datetime.fromisoformat(dt.replace("Z", "+00:00")).strftime("%Y%m%dT%H%M%SZ")

# Function to generate iCalendar event string from a memory trace
def generate_ics_string(trace: dict) -> str:
    """
    Convert a memory trace to iCalendar event format.
    
    Parameters:
    - trace: A dictionary containing memory trace data with fields:
        - id: Unique identifier for the memory
        - timestamp: ISO 8601 datetime string
        - title: Event title
        - content: Detailed description
        - location: Optional location string
    
    Returns:
    - String in iCalendar VEVENT format
    """
    start_dt = datetime.fromisoformat(trace["timestamp"].replace("Z", "+00:00"))
    end_dt = start_dt + timedelta(minutes=60)  # Default 60-minute (1 hour) duration

    summary = trace["title"]
    description = trace["content"]
    location = trace.get("location", "")
    uid = f"{trace['id']}@memorysystem.ai"

    return f"""BEGIN:VEVENT
UID:{uid}
DTSTAMP:{datetime_to_ics(trace['timestamp'])}
DTSTART:{datetime_to_ics(trace['timestamp'])}
DTEND:{datetime_to_ics(end_dt.isoformat())}
SUMMARY:{summary}
DESCRIPTION:{description}
LOCATION:{location}
STATUS:CONFIRMED
END:VEVENT
"""

# Process all memory traces and convert to iCalendar events
print("Converting memory traces to iCalendar format...")
ics_events = [generate_ics_string(trace) for trace in memory_traces["memory"]]

# Create the complete iCalendar content as a string variable for later use
ics_content = "BEGIN:VCALENDAR\nVERSION:2.0\nPRODID:-//MemorySystem//EN\n"
ics_content += "".join(ics_events)
ics_content += "END:VCALENDAR"

# Write the content to a file
output_path = Path("../data/calendar_memory_demo.ics")
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, "w") as f:
    f.write(ics_content)
# Test and print the output of the functions
print("Testing datetime_to_ics function:")
for trace in memory_traces["memory"]:
    print(f"Original timestamp: {trace['timestamp']}")
    print(f"Converted to iCalendar format: {datetime_to_ics(trace['timestamp'])}\n")

print("Testing generate_ics_string function:")
for trace in memory_traces["memory"]:
    ics_string = generate_ics_string(trace)
    print(f"Memory Trace ID: {trace['id']}")
    print("Generated iCalendar Event:")
    print(ics_string)
    print("\n" + "-"*40 + "\n")


# Print the absolute path
print(f"The .ics file is saved to: {output_path.resolve()}")
print("iCalendar file 'calendar_memory_demo.ics' has been created.")
print("The iCalendar content is also stored in the 'ics_content' variable for later use.")

Converting memory traces to iCalendar format...
Testing datetime_to_ics function:
Original timestamp: 2025-04-22T17:00:00Z
Converted to iCalendar format: 20250422T170000Z

Original timestamp: 2025-04-23T17:00:00Z
Converted to iCalendar format: 20250423T170000Z

Original timestamp: 2025-04-24T17:00:00Z
Converted to iCalendar format: 20250424T170000Z

Testing generate_ics_string function:
Memory Trace ID: m101
Generated iCalendar Event:
BEGIN:VEVENT
UID:m101@memorysystem.ai
DTSTAMP:20250422T170000Z
DTSTART:20250422T170000Z
DTEND:20250422T180000Z
SUMMARY:Design JSON Schema for Memory Traces
DESCRIPTION:Finalize the structured schema to capture conversation goals, reflections, and observations.
LOCATION:SF Office
STATUS:CONFIRMED
END:VEVENT


----------------------------------------

Memory Trace ID: m102
Generated iCalendar Event:
BEGIN:VEVENT
UID:m102@memorysystem.ai
DTSTAMP:20250423T170000Z
DTSTART:20250423T170000Z
DTEND:20250423T180000Z
SUMMARY:Initial Demo Success
DESCRIPTION:Successf

### 3. Visualize in Calendar Tools

You can import the generated `.ics` file into most calendar applications.

#### Apple Calendar

1. Open Apple Calendar.
2. In the top menu, click **File > Import...**
3. Select the `calendar_memory_demo.ics` file.
4. Choose a calendar to add the events to.
5. Click **Import**.


<img src="figures/iCalendar2.png" alt="Apple Calendar Import" width="400">

#### Google Calendar 

1. Go to [Google Calendar](https://calendar.google.com).
2. On the left sidebar, click the **gear icon > Settings**.
3. Under **Import & Export**, choose **Import**.
4. Select the `calendar_memory_demo.ics` file.
5. Choose a target calendar and click **Import**.

<img src="figures/google-cal.png" width="400">

### 4. Next Steps

This demo shows how to convert structured memory traces from conversation history into calendar events using the `.ics` format and familiar calendar APIs.

To extend this workflow, consider:

- Adding vector embeddings to each trace for similarity-based retrieval
- Using embeddings to retrieve relevant memory during agent prompting
- Exploring the companion notebook: [`memory_embedding_demo.ipynb`](../notebooks/memory_embedding_demo.ipynb)

For more customization in API integration, see:

- [RFC 5545 – iCalendar Specification](https://datatracker.ietf.org/doc/html/rfc5545)  
- [Google Calendar API Documentation](https://developers.google.com/calendar)
